1. Loading dependencies.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import os
import sys
from windrose import WindroseAxes

2. Summary Statistics & Missing-Value Report

In [ ]:
df = pd.read_csv('..\\data\\sierraleone-bumbuna.csv', parse_dates=['Timestamp'])  # add parse_dates parameter
# 1. Summary statistics for all numeric columns
numeric_summary = df.describe()

# 2. Count of missing values in each column
missing_counts = df.isna().sum()

# 3. Identifying columns with >5% missing values
threshold = 0.05 * len(df)
high_na_columns = missing_counts[missing_counts > threshold].index.tolist()

# Printing the results
print("Summary Statistics (Numeric Columns):\n", numeric_summary)
print("\nMissing Value Counts:\n", missing_counts)
print("\nColumns with >5% Missing Values:\n", high_na_columns)

3. missing values

In [ ]:
# Checking for missing values in key columns
key_columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']
missing_counts = df[key_columns].isna().sum()
print("Missing Values:\n", missing_counts)


4. Incorrect entries Solar Irradiance (GHI, DNI, DHI)

In [ ]:
# Negative values (physically impossible)
invalid_ghi = df[df['GHI'] < 0]
invalid_dni = df[df['DNI'] < 0]
invalid_dhi = df[df['DHI'] < 0]

# Unrealistically high values (adjust thresholds)
high_ghi = df[df['GHI'] > 1200]  # Max ~1200 W/m²
high_dni = df[df['DNI'] > 1300]   # Max ~1300 W/m²
high_dhi = df[df['DHI'] > 400]    # Max ~400 W/m²

print("Invalid GHI values:\n", invalid_ghi)
print("Invalid DNI values:\n", invalid_dni)
print("Invalid DHI values:\n", invalid_dhi)
print("High GHI values:\n", high_ghi)
print("High DNI values:\n", high_dni)
print("High DHI values:\n", high_dhi)

5. Incorrect entries Sensor Readings (ModA, ModB)

In [ ]:
# Negative or stuck values
invalid_moda = df[(df['ModA'] < 0) | (df['ModA'] > 1000)]  # Adjust max threshold
stuck_moda = df['ModA'].nunique() == 1  # No variation → sensor failure?

# Compare ModA vs. ModB for consistency
discrepancy = df[abs(df['ModA'] - df['ModB']) > 100]  # Large differences

6. Incorrect entries Wind Speed (WS, WSgust)

In [ ]:
# Negative or illogical values (gust < avg speed)
invalid_ws = df[df['WS'] < 0]
gust_errors = df[df['WSgust'] < df['WS']]  # Gust must be ≥ WS


7. Fixing Values

In [ ]:
valid_rows = df[
    # Solar Irradiance: All conditions must hold
    (df["GHI"] > 0) & (df["GHI"] < 1200) &
    (df["DNI"] > 0) & (df["DNI"] < 1300) &
    (df["DHI"] > 0) & (df["DHI"] < 400) &
    
    # Wind Speed: All conditions must hold
    (df["WS"] > 0) &
    (df["WSgust"] > 0) &
    (df["WSgust"] > df["WS"])
]

invalid_rows = df[~df.index.isin(valid_rows.index)]
# Print the number of valid and invalid rows   
print(f"Invalid rows: {len(invalid_rows)}")

#df = valid_rows.copy()


8. Outlier Detectiong

In [ ]:
# Compute Z-scores for the selected columns
z_scores = df[key_columns].apply(stats.zscore)

# Identify rows with |Z| > 3
outlier_mask = (np.abs(z_scores) > 3).any(axis=1)
outlier_count = outlier_mask.sum()
print(f"\nTotal Outlier Rows: {outlier_count}")

# Drop outliers
df_cleaned = df[~outlier_mask].copy()

9. Drop or impute (median) missing values in key columns

In [ ]:
# Fill missing values with median
df_cleaned[key_columns] = df_cleaned[key_columns].fillna(df_cleaned[key_columns].median())

# Verify if all missing values are handled
print("\nRemaining Missing Values:\n", df_cleaned[key_columns].isna().sum())

10. Export cleaned DataFrame

In [ ]:
output_path = f"..\\data\\sierraleone_clean.csv"

# Ensure 'data/' directory exists
os.makedirs('..\\data', exist_ok=True)

# Export cleaned data
df_cleaned.to_csv(output_path, index=False)
print(f"\nCleaned data exported to: {output_path}")

11. Time Series Analysis

In [ ]:
# Convert Timestamp to datetime and set as index

df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df.set_index('Timestamp', inplace=True)

# Plot GHI, DNI, DHI, Tamb in subplots
fig, axes = plt.subplots(4, 1, figsize=(14, 12), sharex=True)

# Solar Irradiance
df['GHI'].plot(ax=axes[0], color='blue', title='GHI (W/m²)')
df['DNI'].plot(ax=axes[1], color='orange', title='DNI (W/m²)')
df['DHI'].plot(ax=axes[2], color='green', title='DHI (W/m²)')

# Ambient Temperature
df['Tamb'].plot(ax=axes[3], color='red', title='Tamb (°C)')

plt.tight_layout()
plt.show()

In [ ]:
df = pd.read_csv('..\\data\\sierraleone-bumbuna.csv', parse_dates=['Timestamp'])  # add parse_dates parameter
# Convert 'Timestamp' to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Optionally set as index for easier plotting
df.set_index('Timestamp', inplace=True)

plt.figure(figsize=(14, 10))

# Plot each variable
plt.plot(df.index, df['GHI'], label='GHI')
plt.plot(df.index, df['DNI'], label='DNI')
plt.plot(df.index, df['DHI'], label='DHI')
plt.plot(df.index, df['Tamb'], label='Tamb')

plt.xlabel('Timestamp')
plt.ylabel('Values')
plt.title('Solar Irradiance & Ambient Temperature Over Time')
#plt.legend()
plt.show()

In [ ]:
# Monthly average values
monthly_avg = df.resample('M').mean()

monthly_avg[['GHI', 'DNI', 'DHI', 'Tamb']].plot(kind='bar', figsize=(14, 8))
plt.title('Monthly Average Solar Irradiance & Temperature')
plt.ylabel('Average Value')
plt.show()


In [ ]:
# Hourly average (aggregated across all days)
hourly_avg = df.groupby(df.index.hour).mean()

hourly_avg[['GHI', 'DNI', 'DHI', 'Tamb']].plot(kind='bar', figsize=(14, 8))
plt.title('Average Hourly Solar Irradiance & Temperature')
plt.xlabel('Hour of Day')
plt.ylabel('Average Value')
plt.show()

In [ ]:

# 1. Group by Cleaning flag and calculate means
cleaning_groups = df.groupby('Cleaning')[['ModA', 'ModB']].mean()

# 2. Plot the results
plt.figure(figsize=(10, 6))

# Bar positions
x = [0, 1]  # 0=Before cleaning (0), 1=After cleaning (1)
width = 0.35

# Plot ModA and ModB bars side by side
bars_moda = plt.bar([pos - width/2 for pos in x], 
                   cleaning_groups['ModA'], 
                   width, label='ModA', color='skyblue')
bars_modb = plt.bar([pos + width/2 for pos in x], 
                   cleaning_groups['ModB'], 
                   width, label='ModB', color='orange')

# Customize plot
plt.xticks(x, ['Before Cleaning (0)', 'After Cleaning (1)'])
plt.ylabel('Average Irradiance (W/m²)')
plt.title('Average ModA & ModB Performance: Pre vs. Post Cleaning')
plt.legend()

# Add value labels on top of bars
for bars in [bars_moda, bars_modb]:
    for bar in bars:
        height = bar.get_height()
        plt.text(bar.get_x() + bar.get_width()/2., height,
                f'{height:.1f}',
                ha='center', va='bottom')

plt.grid(True, alpha=0.5)
plt.tight_layout()
plt.show()
# 3. Calculate total cleaning done
cleaning = df[df['Cleaning'] == 1].sum()
print(f"Total cleaning done:\n {cleaning}")

In [ ]:
# Select columns of interest
corr_columns = ['GHI', 'DNI', 'DHI', 'TModA', 'TModB']
corr_matrix = df[corr_columns].corr()

# Plot heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap: Solar Irradiance & Module Temperatures')
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(18, 5))

sns.scatterplot(data=df, x='WS', y='GHI', ax=axs[0])
axs[0].set_title('WS vs GHI')

sns.scatterplot(data=df, x='WSgust', y='GHI', ax=axs[1])
axs[1].set_title('WSgust vs GHI')

sns.scatterplot(data=df, x='WD', y='GHI', ax=axs[2])
axs[2].set_title('WD vs GHI')


fig, axs = plt.subplots(1, 3, figsize=(12, 5))
sns.scatterplot(data=df, x='RH', y='GHI', ax=axs[0])
axs[0].set_title('RH vs GHI')

sns.scatterplot(data=df, x='RH', y='Tamb', ax=axs[1])
axs[1].set_title('RH vs Tamb')

sns.scatterplot(data=df, x='GHI', y='Tamb', ax=axs[2])
axs[2].set_title('GHI vs Tamb')


plt.tight_layout()
plt.show()


plt.tight_layout()
plt.show()


In [ ]:
# Drop NaNs if needed
df_hist = df[['GHI', 'WS']].dropna()

# Histogram for GHI
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
sns.histplot(df_hist['GHI'], bins=30, kde=True, color='orange')
plt.title("Histogram of GHI")

# Histogram for WS
plt.subplot(1, 2, 2)
sns.histplot(df_hist['WS'], bins=30, kde=True, color='skyblue')
plt.title("Histogram of Wind Speed (WS)")

plt.tight_layout()
plt.show()

In [ ]:
# Drop rows with missing values in relevant columns
df_bubble = df[['GHI', 'Tamb', 'RH']].dropna()

# Create the bubble chart
plt.figure(figsize=(10, 6))
plt.scatter(
    df_bubble['GHI'], 
    df_bubble['Tamb'], 
    s=df_bubble['RH'],        # Bubble size = Relative Humidity
    c=df_bubble['RH'],        # Color (optional, also RH here)
    cmap='coolwarm', 
    alpha=0.6, 
    edgecolors='w', 
    linewidth=0.5
)

plt.title('Bubble Chart: GHI vs. Tamb (Bubble size = RH)')
plt.xlabel('GHI (Global Horizontal Irradiance)')
plt.ylabel('Tamb (Ambient Temperature)')
plt.colorbar(label='Relative Humidity (RH)')
plt.grid(True)
plt.tight_layout()
plt.show()
